In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

In [107]:
df = pd.read_csv("../datasets/titanic/train.csv")

In [108]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [109]:
cut = df[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp',
       'Parch']].copy()

In [110]:
second = df.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)

In [111]:
cut.describe()

,Survived,Pclass,Age,SibSp,Parch
count,891.000000,891.000000,714.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594
std,0.486592,0.836071,14.526497,1.102743,0.806057
min,0.000000,1.000000,0.420000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000
50%,0.000000,3.000000,28.000000,0.000000,0.000000
75%,1.000000,3.000000,38.000000,1.000000,0.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000


In [112]:
cut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
dtypes: float64(1), int64(4), object(1)
memory usage: 41.8+ KB


In [113]:
s = pd.Series([1, np.nan, 3, 5, np.nan, 9])

In [114]:
# cut["Age"].fillna(cut["Age"].mean(), inplace=True)

In [115]:
cut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
dtypes: float64(1), int64(4), object(1)
memory usage: 41.8+ KB


In [116]:
droped = cut.loc[cut["Age"].isna()]

In [117]:
cut.dropna(inplace=True)

In [118]:
board = cut.groupby(["Survived", "Pclass", "Sex"])[["Age"]].mean()
board.reset_index(inplace=True)

In [119]:
def get_age(row):
    
    item = board.loc[(board["Survived"]==row["Survived"]) & (board["Pclass"]==row["Pclass"]) & (board["Sex"]==row["Sex"])]
    row["Age"] = item.iat[0,-1]
    return row

In [120]:
board.loc[(board["Survived"]==0) & (board["Pclass"]==1) & (board["Sex"]=="male")]

,Survived,Pclass,Sex,Age
1,0,1,male,44.581967


In [121]:
droped = droped.apply(get_age, axis=1)

In [122]:
total = pd.concat([cut, droped])

In [123]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 888
Data columns (total 6 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
dtypes: float64(1), int64(4), object(1)
memory usage: 48.7+ KB


In [124]:
# df.interpolate(inplace=True)

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [138]:
le = LabelEncoder()

In [139]:
total["Sex"] = le.fit_transform(total["Sex"])

In [141]:
le.classes_

array(['female', 'male'], dtype=object)

In [142]:
X = total.drop(["Survived"], axis=1)

In [143]:
y = total["Survived"]

In [144]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [169]:
model = DecisionTreeClassifier(max_depth=15)

In [170]:
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=15,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [171]:
model.score(X_test, y_test)

0.8203389830508474

In [176]:
def entropy(col):
    vals, counts = np.unique(col, return_counts=True)
    
    acc = 0
    probs = counts / len(col)
    for prob in probs:
        acc += prob * np.log2(prob)
        
    return - acc

In [177]:
entropy(total["Survived"])

0.9607079018756469

In [180]:
def infogain(X, col_name, y):
    
    pivot = X[col_name].mean()
    left = y.loc[X[col_name] < pivot]
    right = y.loc[X[col_name] >= pivot]
    
    l_entropy = entropy(left)
    r_entropy = entropy(right)
    tot_entropy = entropy(y)
    p_left = len(left)/len(y)
    p_right = len(right)/len(y)
    
    return tot_entropy - p_left*l_entropy - p_right*r_entropy


In [181]:
for col in X.columns:
    print("ingogain for ", col, infogain(X, col, y))

ingogain for  Pclass 0.07579362743608165
ingogain for  Sex 0.2176601066606143
ingogain for  Age 0.0008052404301705507
ingogain for  SibSp 0.009584541813400127
ingogain for  Parch 0.015380754493137666
